# Ridge Regression with Gradient Descent

### Author: Juan Solorio

-----

# Overview
In this exercise, I will implement a first version of my own gradient descent algorithm to
solve the ridge regression problem. I will keep improving and extending this gradient descent optimization algorithm. In this notebook, I will implement a basic version of the algorithm.

## Objectives
- Mathematically define _Objective Function_ for Ridge Regression ($F\beta$)
    - Compute gradiant $\nabla F$
- Create functions for algorithm:
    - Objective function
    - Gradient function
    - Gradient Descent funtion
- Observe (plot) the convergence in terms of the function values and the gradients  
    and tune for the optimal hyperparameters for step-size ($\eta$) and normalization ($\lambda$)
- Compare to _sklearn_


## Environment Setup - *Importing Libraries*

In [1]:
# needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Ridge

%matplotlib inline
plt.style.use('ggplot')

In [2]:
X_train = np.array([1,2,3,4])
arr2 = np.array([1,3,6,9])

In [5]:
np.linalg.norm(arr1), np.sqrt(arr1 @ arr1), arr1.dot(arr1)

(5.477225575051661, 5.477225575051661, 30)

# Background and Theory

We start by defining the ___Linear Regression___ as a supervised learning algorithm and we write the mathematical algorithm as:
$$
y = mx + b
$$
> y - target value  
m - slope  
x - predictor variable  
b - intercept

where $m$ and $b$ are the variables the algorithm will try to predict from the data.

We can generalize this equation for multiple variables and write our _cost function_ to optimize our algorithm as:

$$
F(\beta) = \frac{1}n\sum_{i=1}^n(y_i - h(x_i))^2 \\
\\
\\
h(x_i) = x_1\beta_1 + x_2\beta_2 + ... + x_{i,j}\beta_j
$$

Where $h(x_i)$ is the predicted value for the function, $\beta$ are the weights for the individual variables, and $y_i$ is the target value of the data.

We can finalize this by writing the equation in the expanded form:
$$
F(\beta) = \frac{1}n\sum_{i=1}^n(y_i - \sum_{j=1}^dx_{ij}\beta_j)^2
$$

## Lasso Regression

Linear Regression treats all the features equally and finds unbiased weights to minimizes the cost function.
In _Ridge Regression_ , there is an addition of l2 penalty ( square of the magnitude of weights ) in the cost function of Linear Regression:


$$
\lambda\|\beta\|^2_2

$$

Where $\lambda$ is a hyperparameter to be tuned and this is done so that the model does not overfit the data.  

We can write the final objective equation as:


$$
F(\beta) = \frac{1}n\sum_{i=1}^n(y_i - \sum_{j=1}^dx_{ij}\beta_j)^2 + \lambda\sum_{j=1}^d\beta_j^2
$$


## Gradient Descent
Gradient descent is an optimization algorithm used to minimize some function by iteratively moving in the direction of steepest descent as defined by the negative of the gradient. In machine learning, we use gradient descent to update the parameters or weights ($\beta$) of our model.

In order to  minimize with respect to $\beta$, we want to take the derivative of $F(\beta)$ and set it equal to zero. 

The derivative for $n=1$ and $d=1$ yields:
\begin{equation}
\frac{dF}{d\beta} = \frac{2}n(y - x\beta) + 2\lambda\beta
\end{equation}

and for $n>1$ and $d>1$


$$
\frac{\partial F}{\partial \beta_j} = \frac{\partial}{\partial \beta_j} (\frac{1}n\sum_{i=1}^n(y_i - x^T_{i}\beta_j)^2 + \lambda\|\beta\|^2_2)
$$
$$
= \frac{1}n(\sum_{i=1}^n \frac{\partial}{\partial \beta_j} (y_i - x^T_{i}\beta_j)^2 + \frac{\partial}{\partial \beta_j} \lambda\|\beta\|^2_2)
$$
$$
= -\frac{2}n(\sum_{i=1}^n x_{ij} (y_i - x^T_{i}\beta_j)) + 2\lambda\sum_{j=1}^d\beta_j
$$



In Matrix terms, $F(\beta)$ can be interpreted as:


$$
F(\beta) = \frac{1}{n}<Y - X\beta,Y-X\beta> + \lambda\|\beta\|^2_2
$$


Taking the derivative of the matrix form of $F(\beta)$ leads us to the following:


$$
\nabla F = \frac{\partial}{\partial\beta} F= - \frac{2}{n}X^T(y + X\beta) + 2\lambda\beta
$$


Which is also known as the _gradient_ of the _objective function_ $F$.


# Algorithm Functions

* Objective function for Ridge Regression $F(\beta)$:

>$$
F(\beta) = \frac{1}n\sum_{i=1}^n(y_i - X^T_{ij} \cdot \beta_j)^2 + \lambda\|\beta_j\|^2_2
$$

In [7]:
def obj_fx(beta,lamda,X,y):
    """
    Linear regression with Ridge penalty L1:
    F(b) = 1/n sum((y - x dotproduct b)^2) + lamda*norm(b)^2
    
    Parameters
    ----------
    beta : arr
        array of values for weights
    lamda : int
        interger value for normalization parameter
    X : arr
        array of features from data
    y : arr
        array of target values from data

    Returns
    -------
    int
        computation of the objective function

    """
    
    # dot product can be accomplish by 'numpy_arrayA @ numpy_arrayB' or 'numpy_arrayA.dot(numpy_arrayB)'
    return 1/len(y) * sum((y - X @ beta)**2) + lamda*np.linalg.norm(beta)

* Gradient of objective funtion - $\nabla F$:
>$$
\nabla F = \frac{\partial}{\partial\beta} F= - \frac{2}{n}X^T(y + X\beta) + 2\lambda\beta
$$

In [8]:
def gradient_fx(beta,lamda,X,y):
    """
    Computes gradient of the Linear regression with Ridge penalty L1 function:
    grad F(b) = -2/n (x.T dotproduct (y - x dotproduct b)) + 2*lamda*b
    
    Parameters
    ----------
    beta : arr
        array of values for weights
    lamda : int
        interger value for normalization parameter
    X : arr
        array of features from data
    y : arr
        array of target values from data

    Returns
    -------
    int
        computation of the gradient of Ridge regression objective function

    """
    return (-2/len(y)) * X.T @ (y - X @ beta) + 2*lamda*beta 

* Gradient Descent Algorith:
>`Gradient Descent algorithm with fixed constant step-size  
__input__  step-size $\eta$  
__initialization__ $\beta_0 = 0$  
__repeat for__ t = 0, 1, 2, . . .  
    $\beta_{t+1} = \beta_t − \eta \nabla F(\beta_t)$  
__until__ the stopping criterion $\|\nabla F\| \leq \epsilon$ is satisfied.


In [9]:
def gradient_descent(beta_init, eta ,lamda,X,y,epsilon=0.005):
    """
    Computes gradient descent with a fixed step size eta and stopping condition norm gradient F < epislon
    
    Parameters
    ----------
    beta_init : arr
        array of values for weights as starting point
    eta : int
        interger value for step size parameter
    lamda : int
        interger value for normalization parameter
    X : arr
        array of features from data
    y : arr
        array of target values from data
    epsilon : int
        interger value for stopping parameter condition, defaul = 0.005

    Returns
    -------
     beta_vals: Matrix 
         Estimated betas at each iteration, with the most recent values in the last row
    """
    # gradient calculation for starting values
    gradient = gradient_fx(beta,lamda,X,y)
    # list to save beta values 
    beta_vals = [beta_init]
    # loop for stopping criterion epsilon
    while np.linalg.norm(gradient) > epsilon:
        # updating values for beta and gradient
        beta = beta - eta*gradient
        gradient = gradient_fx(beta,lamda,X,y)
        
        # appending values
        beta_vals.append(beta)
    return np.array(beta_vals)